In [2]:
import os
import sys
import importlib
import pickle
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
from keras import layers
from tensorflow.keras.models import Sequential
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras.backend as K
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.compat.v1 import Session, placeholder

In [3]:
#tf.config.optimizer.set_jit(True)

notebooks_dir = os.path.abspath('')
proj_dir = os.path.dirname(notebooks_dir)
raw_data_dir = os.path.join(proj_dir, 'raw_data')
src_dir = os.path.join(proj_dir, 'src')

# Add src_dir to the system path to import helper file
sys.path.insert(1, src_dir)
#import cutpaste_helper
#importlib.reload(cutpaste_helper)

In [4]:
def makeWindows(image_dict,num_windows, width, height):
    '''
    Parameters:
    image_dict: dictionary containing arrays that represent images
    num_windows: the number of windows wanted
    width: the width of the window
    height: the height of the window
    '''
    #Grab windows from the full images from train images
    window_images = []
    for key in image_dict:
    #The 100 windows per image in a list
        for i in range(0, num_windows):
            #Get the width and height of an image
            dimensions = image_dict[key].shape
            image_w = dimensions[1]
            image_h = dimensions[0]
            #Random coordinates
            random_x_coord = random.randint(0, image_w - width)
            random_y_coord = random.randint(0, image_h-height)
            #Takes window out of a random part of the images at the random coordinates
            window = image_dict[key][random_y_coord:random_y_coord + window_y, random_x_coord:random_x_coord + window_x]
            window_images.append(window)
    window_images = np.array(window_images)
    return window_images

def makeWhiteSquares(window_list):
    #Create test window images dict with white square defects
    defect_window_list = np.copy(window_list)
#Fill defect train window images
    for image in defect_window_list:
        #Random sized white square (2D array full of ones)
        sq_random_x = random.randint(5,25)
        sq_random_y = random.randint(5,25)
        white_square = np.ones((sq_random_y,sq_random_x))
        #Random coordinates for the square
        random_x_coord = random.randint(0,window_x-sq_random_x)
        random_y_coord = random.randint(0,window_y-sq_random_y)
        #Replace the coordinates with white square
        image[random_y_coord:random_y_coord + sq_random_y, random_x_coord: random_x_coord + sq_random_x,0] = white_square
    return defect_window_list

def copyPaste(image):
    copied_image = np.copy(image)
    #Random sized copy-pasted area (2D array)
    sq_random_x = random.randint(5,25)
    sq_random_y = random.randint(5,25)
    #Random coordinates for the square
    random_x_coord = random.randint(0,image.shape[1]-sq_random_x)
    random_y_coord = random.randint(0,image.shape[0]-sq_random_y)
    #Copy the area
    copyArea = image[random_y_coord:random_y_coord + sq_random_y, random_x_coord: random_x_coord + sq_random_x,0]
    #Paste coordinates
    random_x_coord = random.randint(0,window_x-sq_random_x)
    random_y_coord = random.randint(0,window_y-sq_random_y)
    #Paste the area
    copied_image[random_y_coord:random_y_coord + sq_random_y, random_x_coord: random_x_coord + sq_random_x,0] = copyArea
    return copied_image

def createLabels(image_list, defect_status):
    #Create labels for the images
    labels_list = []
    for image in image_list:
        if defect_status:
            labels_list.append(1)
        else:
            labels_list.append(0)
    labels_list = np.array(labels_list)
    return labels_list
        
def shuffleTwoArrays(image_list, label_list):
    #Shuffles the image and label arrays in the same way
    randomize = np.arange(len(label_list))
    np.random.shuffle(randomize)
    image_list = image_list[randomize]
    label_list = label_list[randomize]
    return (image_list, label_list)

def imageGenerator(normal_image_list, num_normal, num_defects):
    #A generator that generates images
    while True:
        generated_images = []
        generated_labels = []
        for i in range(num_normal):
            random_index = random.randint(0,len(normal_image_list)-1)
            generated_images.append(normal_image_list[random_index])
            generated_labels.append(0)
        for i in range(num_defects):
            random_index = random.randint(0,len(normal_image_list)-1)
            generated_images.append(copyPaste(normal_image_list[random_index]))
            generated_labels.append(1)
        generated_images = np.array(generated_images)
        generated_labels = np.array(generated_labels)
        generated_images, generated_labels = shuffleTwoArrays(generated_images, generated_labels)
        n = generated_images.shape[0]
        yield generated_images, generated_labels.reshape(n, 1)

def valGeneratorImages(test_x, test_y,num_desired):
    while True:
        generated_images = []
        generated_labels = []
        for i in range(num_desired):
            random_index = random.randint(0,len(test_x)-1)
            generated_images.append(test_x[random_index])
            generated_labels.append(test_y[random_index])
        yield generated_images, generated_labels


In [5]:
# Load dictionaries with TEM images
f_name = os.path.join(raw_data_dir, 'test_full_arrays')
#Dict with test image arrays (each key has a value of an array of numbers between 0 and 1)
test_full_arrays = pickle.load(open(f_name, "rb"))
f_name = os.path.join(raw_data_dir, 'train_full_arrays')
#Dict with train image arrays (each key has a value of an array of numbers between 0 and 1)
train_full_arrays = pickle.load(open(f_name, "rb"))

window_x = 118
window_y = 84
num_windows = 100

#Make testing window images and labels
test_window_images = makeWindows(test_full_arrays, num_windows, window_x, window_y)
test_window_labels = createLabels(test_window_images, False)

#Make training window images and labels
train_window_images = makeWindows(train_full_arrays, num_windows ,window_x, window_y)
train_window_labels = createLabels(train_window_images, False)

train_gen = imageGenerator(train_window_images,32,32)
val_gen = imageGenerator(test_window_images,32,32)

In [6]:
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. We'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [7]:
ops.reset_default_graph()
with tf.compat.v1.Session() as test:
    A_prev = tf.compat.v1.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = identity_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.compat.v1.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = ", out[0][1][1][0])

Instructions for updating:
Colocations handled automatically by placer.
out =  [2.9328394 0.        0.        0.7790533 1.6823828 0.2872013]


In [8]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    
    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [9]:
ops.reset_default_graph()
with tf.compat.v1.Session() as test:
    A_prev = tf.compat.v1.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = convolutional_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.compat.v1.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = ", out[0][1][1][0])

out =  [1.500024  0.        0.2182535 0.8360958 3.8878198 0.       ]


In [10]:
def ResNet50(input_shape = (84, 118, 1), classes = 1):   
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL.
    X = AveragePooling2D((2, 2), name='avg_pool')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [11]:
model = ResNet50(input_shape = (window_y, window_x, 1), classes = 1)

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
val_x, val_y = next(val_gen)
print(val_x, val_y)

[[[[0.32308882]
   [0.4200462 ]
   [0.50798005]
   ...
   [0.20819703]
   [0.18378045]
   [0.20421445]]

  [[0.33476683]
   [0.4298741 ]
   [0.5173133 ]
   ...
   [0.21721634]
   [0.1934965 ]
   [0.21538611]]

  [[0.37595317]
   [0.46698534]
   [0.55244267]
   ...
   [0.25127158]
   [0.23021223]
   [0.25539604]]

  ...

  [[0.45528555]
   [0.54335135]
   [0.6238579 ]
   ...
   [0.32007527]
   [0.30604202]
   [0.3344149 ]]

  [[0.38047203]
   [0.4708567 ]
   [0.5561054 ]
   ...
   [0.25483313]
   [0.2340524 ]
   [0.2597443 ]]

  [[0.33548668]
   [0.43040153]
   [0.5178157 ]
   ...
   [0.21767707]
   [0.19401212]
   [0.21606033]]]


 [[[0.3415306 ]
   [0.3735868 ]
   [0.40466115]
   ...
   [0.22653338]
   [0.25533372]
   [0.28651536]]

  [[0.34778214]
   [0.38807973]
   [0.4275223 ]
   ...
   [0.21688503]
   [0.24587685]
   [0.28058398]]

  [[0.35443723]
   [0.40212876]
   [0.44896033]
   ...
   [0.20840205]
   [0.23801635]
   [0.2762184 ]]

  ...

  [[0.33547944]
   [0.34994742]
   [0.3

In [15]:
history = model.fit(
  train_gen,
  validation_data = (val_x, val_y),
  validation_batch_size = int(len(val_x)*.5),
  epochs = 100,
  steps_per_epoch=len(train_window_images)//64,
)

Epoch 1/100
175/175 [==============================] - 1238s 7s/step - loss: 0.7546 - accuracy: 0.5006 - val_loss: 0.6936 - val_accuracy: 0.5000
Epoch 2/100
 23/175 [==>...........................] - ETA: 19:06 - loss: 0.7117 - accuracy: 0.4980

KeyboardInterrupt: 